# Задание 6. Классификация новостей

### Данные
Данные в [архиве](https://drive.google.com/file/d/15o7fdxTgndoy6K-e7g8g1M2-bOOwqZPl/view?usp=sharing). В нём два файла:
- `news_train.txt` тестовое множество
- `news_test.txt` тренировочное множество

С некоторых новостных сайтов были загружены тексты новостей за период  несколько лет, причем каждая новость принаделжит к какой-то рубрике: `science`, `style`, `culture`, `life`, `economics`, `business`, `travel`, `forces`, `media`, `sport`.

В каждой строке файла содержится метка рубрики, заголовок новостной статьи и сам текст статьи, например:

>    **sport**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею разгромила чехов**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею крупно об...**

In [1]:
!unzip news.zip

unzip:  cannot find or open news.zip, news.zip.zip or news.zip.ZIP.


## Задание 6.1 

Обработать данные, получив для каждого текста набор токенов
Обработать токены с помощью (один вариант из трех):

- pymorphy2
- русского [snowball стеммера](https://www.nltk.org/howto/stem.html)
- [SentencePiece](https://github.com/google/sentencepiece) или [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)
    
    
## Задание 6.2

Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. Можно использовать [gensim](https://radimrehurek.com/gensim/models/word2vec.html) . Продемонстрировать семантические ассоциации. 

## Задание 6.3

Реализовать алгоритм классификации документа по категориям, посчитать точноть на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
- SVM
- наивный байесовский классификатор
- логистическая регрессия
    

## Задание 6.4* 

Реализуйте классификацию с помощью нейросетевых моделей. Например [RuBERT](http://docs.deeppavlov.ai/en/master/features/models/bert.html) или [ELMo](https://rusvectores.org/ru/models/).

In [ ]:
!pip install torch
!pip install sentencepiece
!pip install transformers

In [2]:
import pandas as pd
import numpy as np
import random
import re

# 1

In [7]:
train = pd.read_csv(
    "news_train.txt",
    encoding="utf-8",
    sep="\t",
    header=None,
    names=["topic", "header", "text"],
)
test = pd.read_csv(
    "news_test.txt",
    encoding="utf-8",
    sep="\t",
    header=None,
    names=["topic", "header", "text"],
)

In [8]:
train

,topic,header,text
0,sport,Овечкин пожертвовал детской хоккейной школе ав...,Нападающий «Вашингтон Кэпиталз» Александр Овеч...
1,culture,Рекордно дорогую статую майя признали подделкой,"Власти Мексики объявили подделкой статую майя,..."
2,science,Samsung представила флагман в защищенном корпусе,Южнокорейская Samsung анонсировала защищенную ...
3,sport,С футболиста «Спартака» сняли четырехматчевую ...,Контрольно-дисциплинарный комитет (КДК) РФС сн...
4,media,Hopes & Fears объединится с The Village,Интернет-издание Hopes & Fears объявило о свое...
...,...,...,...
14995,life,Составлен рейтинг лучших европейских пляжей 20...,Опубликован рейтинг лучших европейских пляжей ...
14996,media,В «Снобе» объяснили причину смены формата,Генеральный директор «Сноб медиа» Марина Гевор...
14997,economics,Минфин предложил штрафовать за биткоины на 50 ...,"Минфин разработал законопроект, устанавливающи..."
14998,life,Мэл Гибсон заплатит бывшей подруге 750 тысяч д...,Актер и режиссер Мэл Гибсон выплатит своей быв...


In [9]:
pd.concat([df.topic.value_counts() for df in [train, test]], axis=1)

,topic,topic
sport,2215,423
science,2156,466
media,2111,403
economics,2080,426
culture,2053,426
life,2033,415
forces,1225,245
business,554,90
travel,289,54
style,284,52


In [12]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [13]:
word_re = re.compile(r"(?u)\b\w\w+\b")
train["sentences"] = [
    [morph.parse(word)[0].normal_form for word in word_re.findall(text.lower())]
    for text in train["text"]
]

In [14]:
test["sentences"] = [
    [morph.parse(word)[0].normal_form for word in word_re.findall(text.lower())]
    for text in test["text"]
]


In [ ]:
train.to_csv("news_train.csv", sep=';', index=False)
test.to_csv("news_test.csv", sep=';', index=False)

In [16]:
train[['sentences']]

,sentences
0,"[нападать, вашингтон, кэпиталзти, александр, о..."
1,"[власть, мексика, объявить, подделка, статуя, ..."
2,"[южнокорейский, samsung, анонсировать, защитит..."
3,"[контрольный, дисциплинарный, комитет, кдк, рф..."
4,"[интернет, издание, hopes, fears, объявить, св..."
...,...
14995,"[опубликовать, рейтинг, хороший, европейский, ..."
14996,"[генеральный, директор, сноб, медиа, марина, г..."
14997,"[минфин, разработать, законопроект, устанавлив..."
14998,"[актёр, режиссёр, мэл, гибсон, выплатить, свой..."


# 2

In [17]:
from gensim.models import Word2Vec

In [18]:
train = pd.read_csv("news_train.csv", sep=';')
test = pd.read_csv("news_test.csv", sep=';')
train['sentences'] = train['sentences'].apply(eval)
test['sentences'] = test['sentences'].apply(eval)

In [19]:
sentences = train['sentences'].values

In [20]:
w2v = Word2Vec(sentences=sentences, min_count=1, sg=1, workers=8)


In [21]:
w2v.train(sentences, total_examples=w2v.corpus_count, epochs=10)

(24518546, 26511380)

In [ ]:
w2v.wv.most_similar(positive=['россия'], topn=5)

In [23]:
w2v.wv.most_similar(positive=['спорт'], topn=5)

[('мутко', 0.7447776198387146),
 ('штрбск', 0.7119430899620056),
 ('ватерполистка', 0.7057926654815674),
 ('мамедов', 0.6998485326766968),
 ('росохотрыболовсоюз', 0.6969951391220093)]

In [24]:
w2v.wv.most_similar(positive=["гейтс", "apple"], negative=["microsoft"], topn=5)

[('хаффман', 0.6078692674636841),
 ('безос', 0.590789794921875),
 ('балмера', 0.5819479823112488),
 ('дорси', 0.5778718590736389),
 ('джобс', 0.5760341882705688)]

# 3

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import warnings
warnings.filterwarnings("ignore")

In [26]:
X_train = [' '.join(sentence) for sentence in train['sentences']]
y_train = train['topic']
X_test = [' '.join(sentence) for sentence in test['sentences']]
y_test = test['topic']

In [27]:
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [28]:
log_reg = RandomizedSearchCV(
    LogisticRegression(max_iter=200),
    param_distributions={
        'C': uniform(loc=0, scale=4)
    },
)

naive_bayes = RandomizedSearchCV(
    MultinomialNB(),
    param_distributions={
        'alpha': uniform(loc=0, scale=1.5)
    },
)

linear_svc = RandomizedSearchCV(
    LinearSVC(),
    param_distributions={
        'C': uniform(loc=0, scale=4)
    },
)

In [29]:
log_reg.fit(X_train_tfidf, y_train)

RandomizedSearchCV(estimator=LogisticRegression(max_iter=200),
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f6cc42f85b0>})

In [30]:
pd.DataFrame(log_reg.cv_results_)[["rank_test_score", "param_C", "mean_test_score", "std_test_score"]]

,rank_test_score,param_C,mean_test_score,std_test_score
0,10,0.92263,0.858600,0.001020
1,8,1.386244,0.865600,0.002164
2,2,3.480908,0.874200,0.002083
3,4,2.233487,0.871133,0.001655
4,1,3.846872,0.874800,0.002156
5,5,1.773841,0.868533,0.001984
6,7,1.431081,0.865800,0.001939
7,6,1.623336,0.867267,0.002533
8,9,1.21182,0.863533,0.001996
9,3,2.366143,0.872400,0.001083


In [31]:
naive_bayes.fit(X_train_tfidf, y_train)

RandomizedSearchCV(estimator=MultinomialNB(),
                   param_distributions={'alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f6cc42f8550>})

In [32]:
pd.DataFrame(naive_bayes.cv_results_)[["rank_test_score", "param_alpha", "mean_test_score", "std_test_score"]]


,rank_test_score,param_alpha,mean_test_score,std_test_score
0,8,1.396238,0.779333,0.004614
1,1,0.0888,0.831333,0.005099
2,10,1.490012,0.778000,0.004738
3,6,0.856794,0.789600,0.003803
4,9,1.470523,0.778267,0.004716
5,4,0.822053,0.790600,0.003486
6,2,0.568119,0.797933,0.004187
7,5,0.831156,0.790333,0.003639
8,3,0.683983,0.794733,0.003447
9,7,1.337919,0.780733,0.004454


In [33]:
linear_svc.fit(X_train_tfidf, y_train)

RandomizedSearchCV(estimator=LinearSVC(),
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f6cc42f8940>})

In [34]:
pd.DataFrame(linear_svc.cv_results_)[["rank_test_score", "param_C", "mean_test_score", "std_test_score"]]

,rank_test_score,param_C,mean_test_score,std_test_score
0,3,1.674088,0.880400,0.004791
1,1,1.486577,0.880867,0.004573
2,7,2.755005,0.879600,0.005243
3,8,0.64998,0.878867,0.003110
4,5,2.89511,0.880000,0.004949
5,2,1.052774,0.880533,0.003429
6,4,1.719991,0.880133,0.004920
7,9,0.388823,0.875533,0.002315
8,6,1.889683,0.879667,0.004949
9,10,0.227294,0.871533,0.001694


In [35]:
def print_metrics(y_pred, y_test):
    print(f'Precision: {precision_score(y_pred, y_test, average="weighted"):.4f}')
    print(f'Recall: {recall_score(y_pred, y_test, average="weighted"):.4f}')
    print(f'F1 score: {f1_score(y_pred, y_test, average="weighted"):.4f}')
    print(f'Accuracy: {accuracy_score(y_pred, y_test):.4f}')

In [36]:
for model in [log_reg, naive_bayes, linear_svc]:
    print(f'{model.estimator.__class__.__name__}:')
    y_pred = model.best_estimator_.predict(X_test_tfidf)
    print_metrics(y_pred, y_test)
    print()

LogisticRegression:
Precision: 0.8921
Recall: 0.8880
F1 score: 0.8891
Accuracy: 0.8880

MultinomialNB:
Precision: 0.8777
Recall: 0.8410
F1 score: 0.8546
Accuracy: 0.8410

LinearSVC:
Precision: 0.8911
Recall: 0.8883
F1 score: 0.8893
Accuracy: 0.8883



# 4

In [37]:
from bert_dataset import CustomDataset
from bert_classifier import BertClassifier

In [ ]:
BertClassifier??

In [38]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

In [39]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y_train)

LabelEncoder()

In [40]:
y_train = le.transform(y_train)
y_val = le.transform(y_val)
y_test = le.transform(y_test)

In [52]:
classifier = BertClassifier(
        model_path='cointegrated/rubert-tiny',
        tokenizer_path='cointegrated/rubert-tiny',
        n_classes=10,
        epochs=25,
        model_save_path='/content/bert.pt'
)


Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not i

In [53]:
classifier.preparation(
        X_train=list(X_train),
        y_train=list(y_train),
        X_valid=list(X_val),
        y_valid=list(y_val)
    )

In [54]:
import torch
torch.cuda.is_available()

True

In [55]:
classifier.train()

Epoch 1/25
Train loss 0.8424235538219316 accuracy 0.7621666666666667
Val loss 0.8627203686278857 accuracy 0.8096666666666666
----------
Epoch 2/25
Train loss 0.6362161614356033 accuracy 0.8478333333333333
Val loss 0.7766300271461659 accuracy 0.8373333333333333
----------
Epoch 3/25
Train loss 0.4891479755630959 accuracy 0.8901666666666667
Val loss 0.7865329854830513 accuracy 0.8503333333333333
----------
Epoch 4/25
Train loss 0.3736647630451256 accuracy 0.9201666666666667
Val loss 0.9067220698235712 accuracy 0.8443333333333333
----------
Epoch 5/25
Train loss 0.2787035929519105 accuracy 0.9418333333333333
Val loss 0.9414685972828974 accuracy 0.8533333333333333
----------
Epoch 6/25
Train loss 0.19153934527540453 accuracy 0.9619166666666666
Val loss 1.0670702230931008 accuracy 0.8556666666666667
----------
Epoch 7/25
Train loss 0.14356416054904864 accuracy 0.9724999999999999
Val loss 1.1835921637205729 accuracy 0.8473333333333333
----------
Epoch 8/25
Train loss 0.09266531670554597 accu

In [56]:
texts = list(X_test)
labels = list(y_test)

predictions = [classifier.predict(t) for t in texts]

In [57]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

precision, recall, f1score = precision_recall_fscore_support(labels, predictions,average='macro')[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: 0.8151710445175645, recall: 0.8259785322560239, f1score: 0.8202002229070702


In [58]:
print(accuracy_score(labels, predictions))

0.8523333333333334
